In [7]:
import os, math
import numpy as np
import pandas as pd
import seaborn as sns
import helpers

%matplotlib inline
import time
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn import svm
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_classif, chi2, f_classif
from sklearn.metrics import log_loss

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV


In [2]:
text, variants = helpers.get_training('./input/training_variants', './input/training_text')
#train_full, val_full = train_test_split(variants.merge(text, how='inner', on='ID'))
train_full = variants.merge(text, how='inner', on='ID')

#this is 20% of the labeled data
text, variants = helpers.get_test('./input/training_variants', './input/training_text')
test_full = variants.merge(text, how='inner', on='ID')

print(train_full.shape)
#print(val_full.shape)
print(test_full.shape)

(2656, 5)
(665, 5)


Use TF-IDF to Vectorize the texts， a feature selector, then SVM one-vs-all classification. 

In [3]:
start_time = time.time()

##Pipeline
tfidf = TfidfVectorizer(
    min_df=1, max_features=16000, strip_accents='unicode',lowercase =True,
    analyzer='word', use_idf=True, 
    smooth_idf=True, sublinear_tf=True, stop_words = 'english')
ffilter = SelectKBest(mutual_info_classif, k=200)
#ffilter = SelectKBest(chi2, k=500)
#ffilter = SelectKBest(f_classif, k=500)
#ffilter = TruncatedSVD(n_components=100)
#ffilter = LinearDiscriminantAnalysis(n_components=100)

##Data and labels
y_train = train_full["Class"]-1
X_train = ffilter.fit_transform(tfidf.fit_transform(train_full["Text"]), y_train)

#y_val = val_full["Class"]-1
#X_val = ffilter.transform(tfidf.transform(val_full["Text"]))

y_test = test_full["Class"]-1
X_test = ffilter.transform(tfidf.transform(test_full["Text"]))

#y_train_bi = label_binarize(y_train, classes=range(9))
#y_val_bi = label_binarize(y_val, classes=range(9))
#y_test_bi = label_binarize(y_test, classes=range(9))


In [10]:
##Fitting
parameters = {
    "estimator__C": [10],
    "estimator__kernel": ['linear']
    #"estimator__degree": [2, 3]
}

clf = GridSearchCV(OneVsRestClassifier(svm.SVC(probability=True, class_weight='balanced')), param_grid=parameters, scoring='neg_log_loss', n_jobs=-1)


clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=OneVsOneClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'estimator__C': [10], 'estimator__kernel': ['linear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [11]:
pd.DataFrame.from_dict(clf.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_estimator__C,param_estimator__kernel,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,5.588192,20.083695,0.481928,0.545378,10,linear,"{'estimator__C': 10, 'estimator__kernel': 'lin...",1,0.489865,0.563914,0.465537,0.523998,0.490374,0.548223,0.017978,0.213336,0.011589,0.016419


In [13]:
#Evaluate
y_train_prob = clf.predict_proba(X_train)
y_test_prob = clf.predict_proba(X_test)
print(log_loss(y_train, y_train_prob, eps=1e-15, normalize=True, labels=range(9)))
#print(log_loss(y_val, clf.predict_proba(X_val), eps=1e-15, normalize=True, labels=range(9)))
print(log_loss(y_test, y_test_prob, eps=1e-15, normalize=True, labels=range(9)))

ValueError: The number of classes in labels is different from that in y_pred. Classes found in labels: [0 1 2 3 4 5 6 7 8]

In [ ]:
submission('./2nd_layer_data/train_prob', y_train_prob)
submission('./2nd_layer_data/test_prob', y_test_prob)

In [ ]:
y_train

In [ ]:
helpers.plot_roc_curve(y_test_bi, y_test_prob)

In [8]:
start_time = time.time()


X_submit = ffilter.transform(tfidf.transform(\
    pd.read_csv('./input/test_text', sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])["Text"]))

y_submit_prob = clf.predict_proba(X_submit) 

print("--- %s seconds ---" % (time.time() - start_time))

--- 231.57416605949402 seconds ---


In [9]:
helpers.submission('./2nd_layer_data/true_test_prob', y_submit_prob)
np.save('./2nd_layer_data/X_true_test_tfidf', X_submit)

In [ ]:
X_train_tfidf = np.array(X_train.todense())
X_test_tfidf = np.array(X_test.todense())

In [ ]:
np.save('./2nd_layer_data/X_train_tfidf', X_train_tfidf)
np.save('./2nd_layer_data/X_test_tfidf', X_test_tfidf)
np.save('./2nd_layer_data/y_train_tfidf', y_train_bi)
np.save('./2nd_layer_data/y_test_tfidf', y_test_bi)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
1.10161488021
1.19078777039